### POS Tagging: Agenda
* <a href="#section1">What are parts of speech? Why are they useful?</a>
* <a href="#section2">How do you use them with SpaCy?</a>
* <a href="#section3">How do we infer them?</a>
* <a href="#section4"> How do we learn them with SpaCy?</a>


In [14]:
def from_scratch():
    !pip install spacy >> ~/spacy.log
    !python -m spacy download en >> ~/spacy.log
    !jupyter nbextension enable --py --sys-prefix widgetsnbextension

    
from_scratch()

In [18]:
from functools import wraps

def as_list(f):
    @wraps(f)
    def wrapper(*args, **kwargs):
        return list(f(*args, **kwargs))
    
    return wrapper



In [20]:
map_(lambda x: x + 1, range(10))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [22]:
import pandas as pd
import spacy
from spacy.matcher import Matcher
from spacy.attrs import POS
from spacy.en import English
import matplotlib.pyplot as plt
from functools import partial
import nltk
from operator import itemgetter
from itertools import groupby
from nltk.corpus import brown
from collections import defaultdict, Counter
import numpy as np
from spacy.tokens import Doc
from IPython.display import HTML
import warnings
import pandas as pd
from functools import wraps

def as_list(f):
    @wraps(f)
    def wrapper(*args, **kwargs):
        return list(f(*args, **kwargs))
    
    return wrapper
    


warnings.filterwarnings('ignore')
%matplotlib inline

map_ = as_list(map)
filter_ = as_list(filter)
zip_ = as_list(zip)

def rep_sentences(texts):
    html = []
    for text in texts:
        html.append(rep_sentence(text))
    return HTML("".join(html))

def rep_sentence(text, display_pos = True):
    html_colors = ['SkyBlue'
               ,'red'
               ,'YellowGreen'
               ,'yellow'
               ,'orange'
               ,'pink'
               ,'brown'
               ,'purple'
               , 'CadetBlue'
                ,'DarkKhaki'
                ,'DarkSalmon'
                ,'Gold'    
              ]
    doc = nlp(text)
    n_words = len(doc)
    unique_pos = list(set(map(lambda x: x.pos_, doc)))
    pos_to_color = {i:html_colors[unique_pos.index(i)] for i in unique_pos}
    css = ["<style>.word{font-weight:bold;}</style>"]
    for pos in unique_pos:
        css.append('<style>.{}{{background-color:{};}}</style>'.format(*[pos, pos_to_color[pos]]))
    css = "".join(css)

    html = ["<table width=100%>"]
    html.append(css)
    html.append("<tr>")            
    for i in range(n_words):
        word_string= doc[i].orth_
        html.append("<td><span class='word'>{0}</span></td>".format(word_string))
    html.append("</tr>")
    if display_pos:
        html.append("<tr>")            
        for i in range(n_words):
            pos = doc[i].pos_
            color = pos_to_color[pos]
            html.append("<td><span class='{0}'>{0}</span></td>".format(pos))
        html.append("</tr>")
    html = "".join(html)
    return html



def custom_tag_table(list_of_word_tag_tuples):
    html_colors = ['SkyBlue'
               ,'red'
               ,'YellowGreen'
               ,'yellow'
               ,'orange'
               ,'pink'
               ,'brown'
               ,'MediumPurple'
               , 'CadetBlue'
                ,'DarkKhaki'
                ,'DarkSalmon'
                ,'Gold'    
              ]
    
    n_words = len(list_of_word_tag_tuples)
    words, pos_list = zip(*list_of_word_tag_tuples)
    unique_pos = list(set([pos for pair in pos_list for pos in pair]))
    pos_to_color = {i:html_colors[unique_pos.index(i)] for i in unique_pos}
    css = ["<style>.word{font-weight:bold;}</style>"]
    for pos in unique_pos:
        css.append('<style>.{}{{background-color:{};}}</style>'.format(*[pos, pos_to_color[pos]]))
    css = "".join(css)

    html = ["<table width=100%>"]
    html.append(css)
    for i in range(n_words):
        html.append("<tr>")            
        word_string= words[i]
        html.append("<td><span class='word'>{0}</span></td>".format(word_string))
        row = []
        pos_sublist = pos_list[i]
        for pos in pos_sublist:
            entry = "<span class='{0}'>{0}</span> ".format(pos)
            #print entry
            row.append(entry)
        row = "".join(row)
        html.append("<td>{}</td>".format(row))
        html.append("</tr>")
    return "".join(html)
        
    

def nltk_corpus(corpus_name):
    corpus = getattr(nltk.corpus, corpus_name)
    try:
        corpus.ensure_loaded()
    except:
        nltk.download(corpus_name)
    return corpus

#read nltk corpora
def nltk_reader(corpus_name, limit = None):
    corpus = nltk_corpus(corpus_name)
    fileids = corpus.fileids()
    
    if limit:
        doc_iter = (" ".join([" ".join(j) for j in corpus.sents(fileid)]) for fileid in fileids[:limit])
    else:
        doc_iter = (" ".join([" ".join(j) for j in corpus.sents(fileid)]) for fileid in fileids)
    return doc_iter

universal_tags = [
     ['Open Class Words','ADJ','adjective']
    ,['Open Class Words','ADV','adverb']
    ,['Open Class Words','INTJ','interjection']
    ,['Open Class Words','NOUN','noun']
    ,['Open Class Words','PROPN','proper noun']
    ,['Open Class Words','VERB','verb']
    ,['Closed Class Words','ADP','adposition']
    ,['Closed Class Words','AUX','auxiliary']
    ,['Closed Class Words','CCONJ','coordination conjunction']
    ,['Closed Class Words','DET','determiner']
    ,['Closed Class Words','NUM','numeral']
    ,['Closed Class Words','PART','particle']
    ,['Closed Class Words','PRON','pronoun']
    ,['Closed Class Words','SCONJ','subordinating conjection']
    ,['Other','PUNCT','punctuation']
    ,['Other','SYM','symbol']
    ,['Other','X','other']
]
tag_table = pd.DataFrame(universal_tags, columns = ['Category','Abbrev','Part of Speech'])
tag_table = tag_table.set_index(['Category','Abbrev'])

nltk.download('tagsets')
nltk.download('universal_tagset')
nlp = spacy.load('en')

[nltk_data] Downloading package tagsets to /home/jupyter/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


<a name="section1"></a>

### What are Parts of Speech?

In [5]:
tag_table

Part of Speech
Category           Abbrev                          
Open Class Words   ADJ                    adjective
                   ADV                       adverb
                   INTJ                interjection
                   NOUN                        noun
                   PROPN                proper noun
                   VERB                        verb
Closed Class Words ADP                   adposition
                   AUX                    auxiliary
                   CCONJ   coordination conjunction
                   DET                   determiner
                   NUM                      numeral
                   PART                    particle
                   PRON                     pronoun
                   SCONJ   subordinating conjection
Other              PUNCT                punctuation
                   SYM                       symbol
                   X                          other

In [6]:
sentence1 = 'I get a discount on newspapers.'
sentence2 = 'I discount that newspaper.'

rep_sentences([sentence1, sentence2])

<a name='applications'></a>
### Applications
* Rule based systems:
    * <a href="#qacode">Example of rule based question answering component</a>
* Feature engineering for statistical models
    * <a href="#wordsense">Feature for word disambiguation</a>

<a name="section2"></a>
### Parts of Speech with SpaCy

In [7]:
### Accessing
doc = nlp('I get a discount on newspapers')
tags = {}

for word in doc:
    tags[word.orth_] = {'lemma': word.lemma_, 
                        'pos (coarse)': word.pos_, 
                        'pos (fine)':word.tag_}
pd.DataFrame(tags).T

,lemma,pos (coarse),pos (fine)
I,-PRON-,PRON,PRP
a,a,DET,DT
discount,discount,NOUN,NN
get,get,VERB,VBP
newspapers,newspaper,NOUN,NNS
on,on,ADP,IN


### Exercise: Building word vectors that are Part of Speech specific
Steps:
* get documents
* tokenize the documents, and append the part of speech to each token, e.g. dog|NOUN
* train a word2vec model with gensim
* compare the most similar words of 'back||||VERB' vs 'back||||NOUN' (or other combo)

* Hints:
    * model.wv.vocab contains the vocabulary.
    * using a completely unique join character will make it easier to split later.

In [191]:
from gensim.models import Word2Vec

def return_documents():
    from sklearn.datasets import fetch_20newsgroups
    dataset = fetch_20newsgroups()
    corpus = dataset.data
    return corpus

def tokenize_and_tag_documents(documents, nlp, sep_char="|"):
    pass

def build_model(tokenized_docs):
    pass

def compare_most_similar_words_across_pos(word):
    pass

documents = return_documents()
tokenized_and_tagged_documents = tokenize_and_tag_documents(documents, nlp)
model = build_model(tokenized_and_tagged_documents)
compare_most_similar_words_across_pos('back')

<a name="section3"></a>
### How do we infer parts of speech?

In [13]:
from IPython.display import clear_output, display
from ipywidgets import Button
class reveal(object):
    def __init__(self):
        self.text = 'I was loble to find the effix by klepping the Dongle search engine.'
        self.toggle = Button(description='Toggle POS', )
        self.toggle.on_click(self.toggle_pos)
        self.state = False
        display(self.toggle)
        self.display()
        
    def toggle_pos(self, b):
        self.state = not self.state
        self.display()
        
    def display(self):
        clear_output()
        display(HTML(rep_sentence(self.text, display_pos = self.state)))
        
r = reveal()

I,was,loble,to,find,the,effix,by,klepping,the,Dongle,search,engine,.


### Determinants of Part of Speech:
* Word: some words can only be used in a single way; we can memorize these.
* Word shape: if the first letter is capitalized, its likely a proper noun.
* Neighboring part of speech: there are common patterns, such as noun phrases commonly following a determiner. to the beach



| Feature | Notes | Example|
|------|------|------|
|   Word Identity  | Some words can only be used in a single way; we can memorize these.| "the" -> determiner| 
| Word Shape|Capitalization, dashes,  |"I stayed at the Park Hotel."|
|Neighboring parts of speech|There are common patterns what tags can neighbor others|"to the beach" (noun following determiner)|
|Morphological Structures|Word prefixes and suffixes can rule out certain tag types|"-ly" -> adverb|
|Syntactic Dependencies|Syntax may establish expectations that only certain tags can logically fill|"I was told __" -> adpositional phrase or object entity|
|?|?|?|


<a name="section4"></a>
### Training your own tagger

**Steps** : 
* load <a href="#load_data">**training data**</a>, where each observation is represented as (list_of_words, list_of_tags)
* Look at your dataset. Build a **tag map** mapping from the part of speech tags to the <a href="#universaltagset">universal tagset</a>.
* Decide which **features** to use
    * e.g. pos of previous word, identity of current word, etc...
    * spacy.tagger.N1_cluster, spacy.tagger.N1_pos, etc...
    * which word?
        * N1: Next
        * N0: Current
        * P1: Previous
        * etc...
    * which attribute:
        * prefix
        * tag
        * cluster
        * etc...
* create a **vocabulary** object that consumes: 
    * the tagmap, the features we want to use, and a path from which we can load lexeme data (default english)
    * `spacy.vocab.Vocab`
* create a **statistical model** that consumes the features we want to use
    * `spacy.tagger.TaggerModel`
* create a **tagger** that consumes our vocabulary and our model
    * `spacy.tagger.Tagger`


<a name="load_data"></a>

In [23]:
#load conll2000 corpus
from sklearn.model_selection import train_test_split
import nltk
def nltk_corpus(corpus_name):
    '''returns nltk corpus by name. if not loaded, download.'''
    corpus = getattr(nltk.corpus, corpus_name)
    try:
        corpus.ensure_loaded()
    except:
        nltk.download(corpus_name)
    return corpus


def clean(x):
    if x == '-LCB-':
        return '{'
    elif x=='-RCB-':
        return '}'
    elif x == '-RRB-':
        return ")"
    elif x == '-LRB-':
        return "("
    else:
        return x
    
def conll_to_data():
    corpus = nltk_corpus('conll2000')
    all_data= map_(lambda x: [(clean(i[0]), i[1]) for i in x], corpus.iob_sents())
    all_data = [zip_(*i) for i in all_data]
    return all_data

c2000 = conll_to_data()
training_data, testing_data = train_test_split(c2000, test_size = .1)

In [83]:
from pathlib import PosixPath
modelpath = PosixPath('/home/jupyter/mymodel')
if not modelpath.exists():
    modelpath.mkdir()

In [98]:
from spacy.tagger import *
from spacy.language_data import TAG_MAP
import os
from pathlib import PosixPath



from pathlib import PosixPath
modelpath = PosixPath('/home/jupyter/mymodel')
if not modelpath.exists():
    modelpath.mkdir()
nlp.save_to_directory(modelpath)

features = [
    #current word attributes
    (W_orth,),(W_shape,),(W_cluster,),(W_flags,),(W_suffix,),(W_prefix,),

    #-1 word attributes    
    (P1_pos,),(P1_cluster,),(P1_flags,),(P1_suffix,),

    #-2 word attributes     
    (P2_pos,),(P2_cluster,),(P2_flags,),

    #+1 word attributes    
    (N1_orth,),(N1_suffix,),(N1_cluster,),(N1_flags,),    

    #+2 word attributes    
    (N2_orth,),(N2_cluster,),(N2_flags,),

    #combination attributes
    (P1_lemma, P1_pos),(P2_lemma, P2_pos), (P1_pos, P2_pos),(P1_pos, W_orth)
]

def adjust_tagmap(tagmap):
    tagmap['('] = tagmap['-LRB-']
    tagmap[')'] = tagmap['-RRB-']
    tagmap['{'] = tagmap['-LRB-']
    tagmap['}'] = tagmap['-RRB-']
    tagmap['$'] = {POS: PUNCT}
    return tagmap


def get_lemmatizer():
    return None

def make_tagger(vocab, templates):
    model = spacy.tagger.TaggerModel(templates)
    return spacy.tagger.Tagger(vocab,model)

def get_feature_extractors():
    #return nlp.vocab.lex_attr_getters
    return Language.Defaults.lex_attr_getters

#get requirements

tagmap = adjust_tagmap(TAG_MAP)
feature_extractors = get_feature_extractors()
lemmatizer = get_lemmatizer()
vocab = Vocab.load(MODELPATH, feature_extractors, lemmatizer, tagmap)
statistical_model = spacy.tagger.TaggerModel(features)
tagger = spacy.tagger.Tagger(vocab, statistical_model)

In [94]:
# The untrained tagger is awful...
words = ['You','can','always','learn','more','by','reading']
doc = Doc(vocab, words = words)
tagger(doc)
map_(lambda x: (x.orth_, x.pos_), doc)

[('You', 'PUNCT'),
 ('can', 'PUNCT'),
 ('always', 'PUNCT'),
 ('learn', 'PUNCT'),
 ('more', 'PUNCT'),
 ('by', 'PUNCT'),
 ('reading', 'PUNCT')]

In [99]:
#train
from spacy.scorer import Scorer
from spacy.gold import GoldParse


def predict_cycle(vocab, tagger, data, train=True):
    scorer = Scorer()
    
    for words, tags in training_data:
        #create a document, passing in words to become tokens
        doc = Doc(vocab, words = words)
        tagger(doc)
        gold = GoldParse(doc, tags=tags)   

        
        scorer.score(doc, gold)              
        
        if train:
            #train the model        
            tagger.update(doc, gold)
    return tagger, scorer

def train(vocab, tagger, training_data, testing_data, epochs = 10):
    """
    Parameters
    ----------
    tagger (spacy.tagger.Tagger): 
        The tagger to train.
        
    training_data (list):
        Training data containing words and annotated tags. 
        Should have form: [(word1, word2,...),(tag1, tag2, .....)]
        
    epochs (int):
        number of training iterations
        
    verbose (Bool):
        whether to track and print training accuracy
    """
    
    tagger, pre_train_scorer = predict_cycle(vocab, tagger, training_data, train=False)
    tagger, pre_test_scorer = predict_cycle(vocab, tagger, testing_data, train=False)
    print("Pre Training Accuracy: {0} Pre Test Accuracy {1}".format(pre_train_scorer.tags_acc, 
                                                                    pre_test_scorer.tags_acc))
    
    for train_cycle in range(epochs):
            
        tagger, train_scorer = predict_cycle(vocab, tagger, training_data, train=True)
        tagger, test_scorer = predict_cycle(vocab, tagger, testing_data, train=False)

        print("Iteration {0} Training Accuracy: {1} Test Accuracy {2}".format(train_cycle, 
                                                                              train_scorer.tags_acc, 
                                                                              test_scorer.tags_acc))
        #shuffle data    
        np.random.shuffle(training_data)
    
    tagger.model.end_training()
        
    return tagger

tagger = train(vocab, tagger, training_data, testing_data)

Pre Training Accuracy: 0.0 Pre Test Accuracy 0.0
Iteration 0 Training Accuracy: 91.4423175789149 Test Accuracy 96.0234088494362
Iteration 1 Training Accuracy: 96.69474243410906 Test Accuracy 96.83183672070467
Iteration 2 Training Accuracy: 97.8249134592316 Test Accuracy 98.43841039174693
Iteration 3 Training Accuracy: 98.43969565068376 Test Accuracy 98.75329883127121
Iteration 4 Training Accuracy: 98.84669431401446 Test Accuracy 99.38136203173732
Iteration 5 Training Accuracy: 99.118312369332 Test Accuracy 99.46576070192275
Iteration 6 Training Accuracy: 99.29139390615896 Test Accuracy 99.45119443397196
Iteration 7 Training Accuracy: 99.47133015731569 Test Accuracy 99.1907152894403
Iteration 8 Training Accuracy: 99.56044144360284 Test Accuracy 99.19542790554203
Iteration 9 Training Accuracy: 99.60928128320252 Test Accuracy 99.76522603420503
Iteration 10 Training Accuracy: 99.69753573019844 Test Accuracy 99.74808924838058
Iteration 11 Training Accuracy: 99.7566576412928 Test Accuracy 99

In [88]:
def save_tagger(tagger, model_dir):
    if model_dir is not None:
        tagger.model.dump(str(model_dir / 'pos' / 'model'))
        with (model_dir / 'vocab' / 'strings.json').open('w') as file_:
            tagger.vocab.strings.dump(file_)
            
save_tagger(tagger, modelpath)            

### Appendix

<a name='qacode'></a>
### Example Rule Based QA Component Code

In [ ]:
def get_answer_requirements(token):
    if token.tag_ == 'WRB':
        if token.lower_ == 'where':
            #Where was Star Wars Filmed
            return ['LOCATION']
        elif token.lower_ == 'when':
            #When was Star Wars Filmed
            return ['DATE']
        elif token.lower_ == 'how':
            #How much did Star Wars make?
            if token.nbor().lower_ in ('much', 'many'):
                return ['QUANTITY']

            #How old is star wars?
            elif token.nbor().lower_ in ('long', 'old'):
                return ['DURATION']
            else:
                return False
        elif token.lower() == 'whom':
            #Whom did you see?
            return ['PERSON','ORG']      
        else:
            return False
    elif token.tag_ == 'WP':
        #Asking for Identity
        if token.lower_ in ('who', 'whose'):
            #Who directed Star Wars?
            return ['PERSON','ORG']
        if token.lower_ in ('which','what'):
            #What is Star Wars
            return False 
        else: 
            return False
    elif token.tag_ == 'WDT':
        #asking for a choice among options
        if token.lower_ in ('which','what'):
            #which Star Wars did you like best?
            return [token.nbor().lower_] #return neighbor
        else:
            return False
    else:
        return False

<a href='#applications'>back</a>
<a name="wordsense"></a>
##### Word sense disambiguation

In [131]:
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
for syn in wn.synsets('shower'):
    print(syn, syn.definition())

[nltk_data] Downloading package wordnet to /home/jupyter/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Synset('shower.n.01') a plumbing fixture that sprays water over you
Synset('shower.n.02') washing yourself by standing upright under water sprayed from a nozzle
Synset('shower.n.03') a brief period of precipitation
Synset('shower.n.04') a sudden downpour (as of tears or sparks etc) likened to a rain shower
Synset('exhibitor.n.01') someone who organizes an exhibit for others to see
Synset('shower.n.06') a party of friends assembled to present gifts (usually of a specified kind) to a person
Synset('lavish.v.01') expend profusely; also used with abstract nouns
Synset('shower.v.02') spray or sprinkle with
Synset('shower.v.03') take a shower; wash one's body in the shower
Synset('shower.v.04') rain abundantly
Synset('shower.v.05') provide abundantly with


<a href='#applications'>back</a>